## This jupyter notebook was used to finetune the resnet model for guitar chords. 

In [4]:
import torch
import torchvision.models as models

resnet = models.resnet50(pretrained=True)


c:\Users\slyft\anaconda3\envs\ML2Project\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\slyft\anaconda3\envs\ML2Project\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to C:\Users\slyft/.cache\torch\hub\checkpoints\resnet50-0676ba61.pth
100.0%


In [5]:
num_classes = 14

resnet.fc = torch.nn.Linear(resnet.fc.in_features, num_classes)

In [6]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(resnet.parameters(), lr=0.001, momentum=0.9)


In [7]:
import os
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms

# Define the paths to the training and testing directories
train_dir = 'data/training'  # Path to the folder containing subfolders of training data
test_dir = 'data/test'  # Path to the folder containing subfolders of testing data

# Define the transformations to apply to the images
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Create the train_dataset and validation_dataset
train_dataset = ImageFolder(train_dir, transform=transform)
validation_dataset = ImageFolder(test_dir, transform=transform)


In [8]:
from torch.utils.data import DataLoader

batch_size = 32  # Define your preferred batch size

# Create data loaders for the training and validation sets
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)


In [13]:
from tqdm import tqdm

num_epochs = 15  # Adjust the number of training epochs as needed

for epoch in range(num_epochs):
    running_loss = 0.0  # Initialize the running loss for the epoch
    
    # Wrap the train_dataloader with tqdm for the loading bar
    train_dataloader_with_bar = tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=False)
    
    for images, labels in train_dataloader_with_bar:
        optimizer.zero_grad()
        outputs = resnet(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()  # Accumulate the loss
        
        # Update the loading bar with the current loss value
        train_dataloader_with_bar.set_postfix({'Loss': loss.item()})
        
    epoch_loss = running_loss / len(train_dataloader)  # Calculate the average epoch loss
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}")


Epoch 1/15, Loss: 1.0354


Epoch 2/15, Loss: 0.0385


Epoch 3/15, Loss: 0.0131


Epoch 4/15, Loss: 0.0081


Epoch 5/15, Loss: 0.0073


...

Epoch 11/15, Loss: 0.0019


Epoch 12/15, Loss: 0.0018


Epoch 13/15, Loss: 0.0021


Epoch 14/15, Loss: 0.0015


Epoch 15/15, Loss: 0.0014


In [14]:
correct = 0
total = 0

with torch.no_grad():
    for images, labels in validation_dataloader:
        outputs = resnet(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Validation accuracy: {accuracy}%")


Validation accuracy: 19.435104236718225%


In [15]:
torch.save(resnet.state_dict(), "fretboard_model.pth")
